<a href="https://colab.research.google.com/github/Prathammayur/notabot/blob/main/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2
from tensorflow import keras
from keras.preprocessing.image import image_utils as img
img.img_to_array
img.load_img
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers import Input

#model1 is vgg16 for feature extraction
model1=VGG16(include_top=False)
model1=Model(inputs=model1.inputs, outputs=model1.layers[-2].output)

model = tf.keras.models.load_model('D:/Desktop/best_model.h5')

def preprocess(image):
    img = image.resize((224, 224))
    img = np.array(img)
    img = img / 255.0
    img = img.reshape((1, 224, 224, 3))
    return img

BASE_DIR='D:/Desktop/hack/flickr8k'
# Reading the descriptions.txt file
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    desc_doc = f.read()


#Mapping the descriptions to the images
mapping = {}
for each_desc in (desc_doc.split('\n')):
    tokens = each_desc.split(',')
    if len(each_desc) < 2:
        continue
    image_id, desc_of = tokens[0], tokens[1:]
    image_id = image_id.split('.')[0]
    desc_of = " ".join(desc_of)
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(desc_of)

# Appending all descriptions into a list: Each image with 5 descriptions
img_desc = []
for key in mapping:
    for caption in mapping[key]:
        img_desc.append(caption)


tokenizer = Tokenizer()
tokenizer.fit_on_texts(img_desc)

def generate_caption(image):
    img = preprocess(image)
    caption = '<start> '
    max_len = 35
    

    for i in range(max_len):
        seq = tokenizer.texts_to_sequences([caption])[0]
        seq = tf.keras.preprocessing.sequence.pad_sequences([seq], maxlen=max_len, padding='post')
        preds = model.predict([img, seq])
        word_pred = index_word[np.argmax(preds[0])]
        caption += word_pred + ' '
        if word_pred == '<end>':
            break
    return caption